In [ ]:
# ========================================
# SETUP CELL - Run this first!
# ========================================

# Import required libraries
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import yaml
import subprocess
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import warnings
warnings.filterwarnings('ignore')

# Set up paths
ROOT_DIR = Path('.')
SCRIPTS_DIR = ROOT_DIR / 'accel_tlnd_model' / 'scripts'
INPUTS_DIR = ROOT_DIR / 'accel_tlnd_model' / 'inputs'
RESULTS_DIR = ROOT_DIR / 'accel_tlnd_model' / 'results'
PARAMS_FILE = SCRIPTS_DIR / 'params.yaml'

# ========================================
# UTILITY FUNCTIONS
# ========================================

def load_params():
    """Load parameters from params.yaml"""
    with open(PARAMS_FILE, 'r') as f:
        return yaml.safe_load(f)

def save_params(params):
    """Save parameters to params.yaml"""
    with open(PARAMS_FILE, 'w') as f:
        yaml.dump(params, f, default_flow_style=False)

def save_config(widgets_dict):
    """Save current widget values to params.yaml"""
    config = {
        'my_shp': widgets_dict['file'].value,
        'model_selection': widgets_dict['model'].value,
        'max_dist': widgets_dict['max_dist'].value,
        'project_name': widgets_dict['project'].value,
        'epsg': widgets_dict['epsg'].value,
        'costs_params': {
            'lv_cost_per_meter': widgets_dict['lv_cost'].value,
            'mv_cost_per_meter': widgets_dict['mv_cost'].value,
            'transformer_cost': widgets_dict['transformer_cost'].value
        },
        'outlier_exclusion_case': widgets_dict['outlier'].value,
        'eps_meters': widgets_dict['eps'].value,
        'min_samples': widgets_dict['min_samples'].value,
        'milp_params': {
            'time_limit': widgets_dict['time_limit'].value
        }
    }
    
    save_params(config)
    print("✅ Configuration saved to params.yaml")
    return config

def display_network_metrics(project_name):
    """Display network metrics from the results"""
    result_dir = RESULTS_DIR / project_name
    
    # Look for network metrics file
    metrics_files = list(result_dir.glob('*metrics*')) + list(result_dir.glob('*summary*'))
    
    if metrics_files:
        print(f"\n📊 Network Design Metrics:")
        print("=" * 50)
        
        for metrics_file in metrics_files:
            try:
                with open(metrics_file, 'r') as f:
                    content = f.read()
                    
                # Parse and format the metrics nicely
                lines = content.strip().split('\n')
                for line in lines:
                    if ',' in line and any(char.isdigit() for char in line):
                        # Split on comma and format nicely
                        parts = line.split(',')
                        if len(parts) == 2:
                            metric_name = parts[0].strip()
                            metric_value = parts[1].strip()
                            
                            # Add appropriate emoji for different metrics
                            if 'node' in metric_name.lower():
                                emoji = "🏠"
                            elif 'transformer' in metric_name.lower():
                                emoji = "⚡"
                            elif 'length' in metric_name.lower() or 'distance' in metric_name.lower():
                                emoji = "📏"
                            elif 'cost' in metric_name.lower():
                                emoji = "💰"
                            elif 'time' in metric_name.lower():
                                emoji = "⏱️"
                            else:
                                emoji = "📈"
                            
                            print(f"{emoji} {metric_name}: {metric_value}")
                    else:
                        # Print other lines as-is if they look like headers or important info
                        if line.strip() and not line.startswith('#'):
                            print(f"📋 {line}")
                            
            except Exception as e:
                print(f"⚠️ Error reading metrics from {metrics_file.name}: {e}")
                
        print("=" * 50)
    else:
        print(f"⚠️ No metrics file found in {result_dir}")

def display_network_image(project_name):
    """Display the generated network.png image"""
    result_dir = RESULTS_DIR / project_name
    network_png = result_dir / 'network.png'
    
    if network_png.exists():
        try:
            print(f"\n🖼️ Network Design Visualization:")
            
            fig, ax = plt.subplots(figsize=(14, 10))
            img = mpimg.imread(network_png)
            ax.imshow(img)
            ax.axis('off')  # Hide axes for cleaner image display
            ax.set_title(f"Network Design: {project_name}", fontsize=18, fontweight='bold', pad=20)
            plt.tight_layout()
            plt.show()
            
            print(f"✅ Network visualization displayed from: {network_png}")
            
        except Exception as e:
            print(f"⚠️ Error displaying network image: {e}")
    else:
        print(f"⚠️ Network image not found at: {network_png}")

def get_available_input_files():
    """Get list of available input files"""
    available_files = []
    
    if INPUTS_DIR.exists():
        # Look for shapefiles, parquet files, geojson, gpkg
        for pattern in ['**/*.shp', '**/*.parquet', '**/*.geojson', '**/*.gpkg']:
            for file_path in INPUTS_DIR.glob(pattern):
                # Make path relative to project root
                relative_path = file_path.relative_to(ROOT_DIR)
                available_files.append(str(relative_path))
    
    # Sort files for better organization
    available_files.sort()
    
    # Add a custom option at the beginning
    available_files.insert(0, "-- Select a file or type custom path --")
    
    return available_files

def create_ui_widgets():
    """Create all UI widgets and return them"""
    params = load_params()
    
    # Get available input files
    available_files = get_available_input_files()
    
    # File selection dropdown
    current_file = params.get('my_shp', 'accel_tlnd_model/inputs/example_1/nodes_for_netdesign.shp')
    
    # Set dropdown value - use current file if it's in the list, otherwise use first option
    dropdown_value = current_file if current_file in available_files else available_files[0]
    
    file_dropdown = widgets.Dropdown(
        options=available_files,
        value=dropdown_value,
        description='Quick Select:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )
    
    # File path textarea (copy/paste friendly)
    file_text = widgets.Textarea(
        value=current_file,
        description='File Path:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px', height='60px'),
        placeholder='Type or paste file path here',
        rows=2
    )
    
    # Project name textarea (copy/paste friendly)  
    project_widget = widgets.Textarea(
        value=params.get('project_name', 'test_project'),
        description='Project Name:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px', height='60px'),
        placeholder='Enter project name',
        rows=2
    )
    
    # Connect dropdown and text
    def on_dropdown_change(change):
        if change['new'] != "-- Select a file or type custom path --":
            file_text.value = change['new']
    
    file_dropdown.observe(on_dropdown_change, names='value')
    
    # File widget wrapper
    class FileWidgetWrapper:
        def __init__(self, text_widget):
            self.text = text_widget
        @property
        def value(self):
            return self.text.value.strip()
        @value.setter
        def value(self, val):
            self.text.value = val
    
    file_widget = FileWidgetWrapper(file_text)
    
    # Other widgets
    model_widget = widgets.Dropdown(options=['mst', 'milp'], value=params.get('model_selection', 'mst'), description='Model:')
    max_dist_widget = widgets.IntText(value=params.get('max_dist', 500), description='Max Distance (m):', style={'description_width': 'initial'}, layout=widgets.Layout(width='300px'))
    epsg_widget = widgets.IntText(value=params.get('epsg', 32636), description='EPSG Code:')
    lv_cost_widget = widgets.FloatText(value=params.get('costs_params', {}).get('lv_cost_per_meter', 8), description='LV Cost ($/m):', style={'description_width': 'initial'}, layout=widgets.Layout(width='300px'))
    mv_cost_widget = widgets.FloatText(value=params.get('costs_params', {}).get('mv_cost_per_meter', 20), description='MV Cost ($/m):', style={'description_width': 'initial'}, layout=widgets.Layout(width='300px'))
    transformer_cost_widget = widgets.FloatText(value=params.get('costs_params', {}).get('transformer_cost', 2500), description='Transformer Cost ($):', style={'description_width': 'initial'}, layout=widgets.Layout(width='350px'))
    outlier_widget = widgets.Checkbox(value=params.get('outlier_exclusion_case', False), description='Outlier Exclusion')
    
    # Create collapsible cost parameters section (title will be displayed separately)
    cost_accordion = widgets.Accordion(children=[widgets.VBox([lv_cost_widget, mv_cost_widget, transformer_cost_widget])], 
                                       titles=('Click to expand/collapse',))
    # Use a platform-compatible way to start collapsed
    import IPython
    if 'google.colab' in str(IPython.get_ipython()):
        # In Google Colab, start expanded but user can collapse
        cost_accordion.selected_index = 0
    else:
        # In local Jupyter, start collapsed
        cost_accordion.selected_index = None
    eps_widget = widgets.IntSlider(value=params.get('eps_meters', 100), min=50, max=500, step=10, description='EPS (m):')
    min_samples_widget = widgets.IntSlider(value=params.get('min_samples', 5), min=1, max=20, description='Min Samples:')
    time_limit_widget = widgets.IntText(value=params.get('milp_params', {}).get('time_limit', 60), description='Time Limit (s):')
    
    return {
        'file': file_widget, 'file_dropdown': file_dropdown, 'file_text': file_text,
        'model': model_widget, 'max_dist': max_dist_widget, 'project': project_widget,
        'epsg': epsg_widget, 'lv_cost': lv_cost_widget, 'mv_cost': mv_cost_widget,
        'transformer_cost': transformer_cost_widget, 'cost_accordion': cost_accordion,
        'outlier': outlier_widget, 'eps': eps_widget, 'min_samples': min_samples_widget, 
        'time_limit': time_limit_widget
    }

print("✅ Setup completed successfully!")
print(f"📁 Working directory: {ROOT_DIR.absolute()}")
print("🎯 Now run the Interactive UI cell below!")


✅ Setup completed successfully!
📁 Working directory: /Users/wuyuezi/Desktop/energy_system_design_tools/accel_tlnd_training
🎯 Now run the Interactive UI cell below!


In [ ]:
# ========================================
# INTERACTIVE UI CELL
# ========================================

# Create all UI widgets
widgets_dict = create_ui_widgets()

# Create buttons
refresh_button = widgets.Button(
    description="🔄 Refresh",
    button_style='info',
    layout=widgets.Layout(width='120px')
)

save_button = widgets.Button(
    description="Save Configuration",
    button_style='success',
    icon='save'
)

run_button = widgets.Button(
    description="Run Analysis",
    button_style='primary',
    icon='play'
)

# Create output area
analysis_output = widgets.Output()

# Create conditional display for time limit (only needed if MILP model is selected)
time_limit_container = widgets.VBox([widgets_dict['time_limit']])

def on_model_change(change):
    if change['new'] == 'milp':
        time_limit_container.children = [widgets_dict['time_limit']]
    else:
        time_limit_container.children = []

widgets_dict['model'].observe(on_model_change, names='value')

# Initialize display based on current model selection
if widgets_dict['model'].value == 'milp':
    time_limit_container.children = [widgets_dict['time_limit']]
else:
    time_limit_container.children = []

# Button click handlers
def on_refresh_click(b):
    try:
        files = get_available_input_files()
        widgets_dict['file_dropdown'].options = files
        print(f"✅ Found {len(files)-1} input files")
    except Exception as e:
        print(f"⚠️ Error: {e}")

def on_save_click(b):
    with analysis_output:
        clear_output()
        save_config(widgets_dict)

def on_run_click(b):
    with analysis_output:
        clear_output()
        save_config(widgets_dict)
        print("")
        
        print("🚀 Starting network design analysis...")
        print("This may take a few minutes depending on your data size and model selection.")
        
        try:
            result = subprocess.run(
                [sys.executable, 'main.py'],
                cwd=SCRIPTS_DIR,
                capture_output=True,
                text=True
            )
            
            if result.returncode == 0:
                print("✅ Analysis completed successfully!")
                print("\n📊 Analysis Output:")
                print(result.stdout)
                
                # Display network metrics first
                display_network_metrics(widgets_dict['project'].value.strip())
                
                # Then display the network visualization
                display_network_image(widgets_dict['project'].value.strip())
                
            else:
                print("❌ Analysis failed!")
                print("\n🔍 Error details:")
                print(result.stderr)
                
                # Still try to display metrics and network.png in case they were generated before failure
                print("\n🔍 Attempting to display any generated network results...")
                display_network_metrics(widgets_dict['project'].value.strip())
                display_network_image(widgets_dict['project'].value.strip())
                
        except Exception as e:
            print(f"❌ Error running analysis: {e}")

# Connect buttons to handlers
refresh_button.on_click(on_refresh_click)
save_button.on_click(on_save_click)
run_button.on_click(on_run_click)

# ========================================
# DISPLAY THE UI
# ========================================

display(HTML("<h2>🔧 Energy System Design Tool</h2>"))

display(HTML("<h3>📁 Input Configuration</h3>"))
display(HTML("""
<div style='color:#555; font-size:0.9em; margin-bottom:6px;'>
<b>Supported formats:</b> Shapefile (.shp), GeoJSON (.geojson), GeoPackage (.gpkg), and Parquet (.parquet).<br>
<b>💡 Tip:</b> Use the dropdown to quickly select available files, or copy/paste a custom path in the text field below.
</div>
"""))
display(widgets.HBox([widgets_dict['file_dropdown'], refresh_button]))
display(widgets_dict['file_text'])
display(widgets_dict['project'])
display(widgets_dict['epsg'])

display(HTML("""
<h3>🔧 Model Configuration</h3>
<div style='color:#888; font-size: 0.95em; margin-bottom:8px;'>
<b>Note:</b> <span style='color:#6a8caf;'>
<strong>Suggestion:</strong> For most cases, select <b>mst</b>.<br>
Choose <b>milp</b> only if you have Gurobi installed and want more advanced optimization.
</span>
</div>
"""))
display(widgets_dict['model'])
display(widgets_dict['max_dist'])
display(time_limit_container)

display(HTML("<h3>💰 Cost Parameters</h3>"))
display(HTML("""
<div style='color:#666; font-size:0.9em; margin-bottom:8px;'>
<b>📝 Note:</b> Expand the panel below to modify cost parameters for your network design calculations.<br>
</div>
"""))
display(widgets_dict['cost_accordion'])

display(HTML("<h3>🎯 Outlier Exclusion</h3>"))
display(widgets_dict['outlier'])
display(widgets_dict['eps'])
display(widgets_dict['min_samples'])

display(HTML("<h2>🚀 Run Analysis</h2>"))
display(widgets.HBox([save_button, run_button]))
display(analysis_output)

print("🎉 Interactive UI is ready! Configure your parameters above and click 'Run Analysis'!")
print("📸 Network visualization will appear automatically after analysis completes.")
print("The output files will be saved at 'results/Project Name'")


Textarea(value='accel_tlnd_model/inputs/example_3_parish_42320601_merged_structures.parquet', description='Fil…

Textarea(value='example_project_3', description='Project Name:', layout=Layout(height='60px', width='400px'), …

IntText(value=32636, description='EPSG Code:')

Dropdown(description='Model:', options=('mst', 'milp'), value='mst')

IntText(value=1000, description='Max Distance (m):', layout=Layout(width='300px'), style=DescriptionStyle(desc…

...

Checkbox(value=False, description='Outlier Exclusion')

IntSlider(value=100, description='EPS (m):', max=500, min=50, step=10)

IntSlider(value=3, description='Min Samples:', max=20, min=1)

Output()

🎉 Interactive UI is ready! Configure your parameters above and click 'Run Analysis'!
📸 Network visualization will appear automatically after analysis completes.
The output files will be saved at 'results/Project Name'
